In [0]:
from pyspark.sql.functions import col, to_json, struct
import json

df = spark.table(
    "workspace.sk_paris_olympics_2024.olympics_ft_training"
)
df = df.filter(
    col("instruction").isNotNull() &
    col("response").isNotNull()
)
ft_df = df.selectExpr(
    "named_struct('role','user','content',instruction) as user_msg",
    "named_struct('role','assistant','content',response) as assistant_msg"
)

rows = ft_df.toPandas().to_dict(orient="records")

# Write directly to a Unity Catalog volume
output_path = "/Volumes/workspace/sk_paris_olympics_2024/output/paris2024_ft_conver.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for r in rows:
        obj = {
            "messages": [r["user_msg"], r["assistant_msg"]]
        }
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print("Wrote JSONL to:", output_path)

Wrote JSONL to: /Volumes/workspace/sk_paris_olympics_2024/output/paris2024_ft_conver.jsonl
